In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 最常重复出现的URLs数据
top_urls = [
    "Promotions Site",
    "View As Webpage",
    "Unsubscribe",
    "Box Link",
    "Storefront"
]

# URL在有效天的表现数据
# 根据分析，只有Sunday, Monday, Tuesday, Wednesday有数据
daily_performance = {
    "Promotions Site": [
        {"Day": "Sunday", "AvgClicks": 41.0, "ViewPct": 3.5, "Efficiency": 2.83},
        {"Day": "Monday", "AvgClicks": 43.0, "ViewPct": 4.08, "Efficiency": 2.16},
        {"Day": "Tuesday", "AvgClicks": 68.0, "ViewPct": 5.45, "Efficiency": 1.45},
        {"Day": "Wednesday", "AvgClicks": 19.0, "ViewPct": 2.2, "Efficiency": 1.46}
    ],
    "View As Webpage": [
        {"Day": "Sunday", "AvgClicks": 7.17, "ViewPct": 0.7, "Efficiency": 2.05},
        {"Day": "Monday", "AvgClicks": 6.92, "ViewPct": 0.65, "Efficiency": 1.93},
        {"Day": "Tuesday", "AvgClicks": 5.5, "ViewPct": 0.45, "Efficiency": 1.83},
        {"Day": "Wednesday", "AvgClicks": 6.0, "ViewPct": 0.7, "Efficiency": 2.0}
    ],
    "Unsubscribe": [
        {"Day": "Sunday", "AvgClicks": 1.6, "ViewPct": 0.16, "Efficiency": 1.33},
        {"Day": "Monday", "AvgClicks": 2.55, "ViewPct": 0.24, "Efficiency": 1.6},
        {"Day": "Tuesday", "AvgClicks": 1.0, "ViewPct": 0.1, "Efficiency": 1.0},
        {"Day": "Wednesday", "AvgClicks": 2.0, "ViewPct": 0.2, "Efficiency": 2.0}
    ],
    "Box Link": [
        {"Day": "Sunday", "AvgClicks": 12.4, "ViewPct": 1.16, "Efficiency": 1.63},
        {"Day": "Monday", "AvgClicks": 13.8, "ViewPct": 1.34, "Efficiency": 1.94}
    ],
    "Storefront": [
        {"Day": "Sunday", "AvgClicks": 6.4, "ViewPct": 0.56, "Efficiency": 1.52},
        {"Day": "Monday", "AvgClicks": 6.28, "ViewPct": 0.61, "Efficiency": 1.68}
    ]
}

# 将字典数据转换为DataFrame，用于绘图
def prepare_daily_df():
    rows = []
    for url_short in top_urls:
        for day_data in daily_performance[url_short]:
            rows.append({
                "URL": url_short,
                "Day": day_data["Day"],
                "AvgClicks": day_data["AvgClicks"],
                "ViewPct": day_data["ViewPct"],
                "Efficiency": day_data["Efficiency"]
            })
    return pd.DataFrame(rows)

daily_df = prepare_daily_df()

# 创建热力图数据
# 只使用有数据的日期
heatmap_data = daily_df.pivot_table(
    values='AvgClicks',
    index='URL',
    columns='Day',
    aggfunc='mean'
).fillna(0)

# 创建点击量热力图
fig1 = px.imshow(
    heatmap_data,
    labels=dict(x="Day of Week", y="URL Type", color="Avg Clicks"),
    x=heatmap_data.columns,
    y=heatmap_data.index,
    title="Average Clicks Heatmap",
    color_continuous_scale="Viridis"
)
fig1.update_layout(height=500)

# 创建浏览率热力图
view_heatmap_data = daily_df.pivot_table(
    values='ViewPct',
    index='URL',
    columns='Day',
    aggfunc='mean'
).fillna(0)

fig2 = px.imshow(
    view_heatmap_data,
    labels=dict(x="Day of Week", y="URL Type", color="View %"),
    x=view_heatmap_data.columns,
    y=view_heatmap_data.index,
    title="View Percentage Heatmap",
    color_continuous_scale="Blues"
)
fig2.update_layout(height=500)

# 创建点击效率热力图
efficiency_heatmap_data = daily_df.pivot_table(
    values='Efficiency',
    index='URL',
    columns='Day',
    aggfunc='mean'
).fillna(0)

fig3 = px.imshow(
    efficiency_heatmap_data,
    labels=dict(x="Day of Week", y="URL Type", color="Click Efficiency"),
    x=efficiency_heatmap_data.columns,
    y=efficiency_heatmap_data.index,
    title="Click Efficiency Heatmap",
    color_continuous_scale="Reds"
)
fig3.update_layout(height=500)

# 创建组合热力图 - 显示数据分布
# 创建一个图表，展示每个单元格的样本数
sample_counts = daily_df.groupby(['URL', 'Day']).size().reset_index(name='count')
sample_heatmap = sample_counts.pivot(index='URL', columns='Day', values='count').fillna(0)

fig4 = px.imshow(
    sample_heatmap,
    labels=dict(x="Day of Week", y="URL Type", color="Sample Count"),
    x=sample_heatmap.columns,
    y=sample_heatmap.index,
    title="Data Distribution - Number of Samples per Day/URL",
    color_continuous_scale="Greens"
)
fig4.update_layout(height=500)

# 创建一个 3D 表面图，展示URL在不同日期的表现
# 为了更好的3D效果，我们将创建一个额外的数据集
days_ordered = ['Sunday', 'Monday', 'Tuesday', 'Wednesday']
urls_ordered = top_urls

# 创建一个规则网格
complete_grid = pd.DataFrame([
    {'URL': url, 'Day': day, 'DayIndex': days_ordered.index(day), 'URLIndex': urls_ordered.index(url)}
    for url in urls_ordered
    for day in days_ordered
])

# 合并实际数据
complete_grid = complete_grid.merge(
    daily_df,
    on=['URL', 'Day'],
    how='left'
).fillna(0)

# 对于3D表面图，数据需要是矩阵形式
z_data = complete_grid.pivot_table(
    values='AvgClicks',
    index='URLIndex',
    columns='DayIndex',
    aggfunc='first'
).values

# 创建3D表面图
fig5 = go.Figure(data=[go.Surface(
    z=z_data,
    x=np.array(range(len(days_ordered))),
    y=np.array(range(len(urls_ordered))),
    colorscale='Viridis'
)])

fig5.update_layout(
    title='3D Surface: Average Clicks by URL and Day',
    scene=dict(
        xaxis=dict(title='Day', ticktext=days_ordered, tickvals=list(range(len(days_ordered)))),
        yaxis=dict(title='URL', ticktext=urls_ordered, tickvals=list(range(len(urls_ordered)))),
        zaxis=dict(title='Average Clicks')
    ),
    height=700
)

# 显示图表
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()